In [1]:
import numpy as np
import pandas as pd

In [2]:
def diferencia_cos(t1, t2):

    # Definir los textos a comparar
    texto1 = t1
    texto2 = t2

    # Función para tokenizar y limpiar el texto


    def preprocess(text):
        text = text.lower().split()
        return [word.strip("°") for word in text]


    # Tokenizar y limpiar los textos
    tokens1 = preprocess(texto1)
    tokens2 = preprocess(texto2)

    # Remover palabras repetidas
    lenguaje = set(tokens1 + tokens2)

    # Generar vectores en 0
    v1 = np.zeros(len(lenguaje))
    v2 = np.zeros(len(lenguaje))


    # Contabilizar palabras en cada texto
    i = 0
    for token in lenguaje:
        v1[i] = tokens1.count(token)
        v2[i] = tokens2.count(token)
        i += 1

    acum_punto = 0
    acum_v1 = 0
    acum_v2 = 0
    for i in range(len(v1)):
        acum_punto += v1[i] * v2[i]
        acum_v1 += v1[i] * v1[i]
        acum_v2 += v2[i] * v2[i]

    sim = acum_punto / (np.sqrt(acum_v1) * np.sqrt(acum_v2))

    return sim


In [48]:
data = pd.read_csv('nuevo_labels.csv')


In [49]:
data['ResulComp'] = data.apply(
    lambda row: diferencia_cos(row['sub1'], row['sub2']), axis=1)


In [50]:
data


,sub1,sub2,problem,verdict,ResulComp
0,//package codeforces;\nimport java.io.PrintWri...,import java.util.*;\n// import java.lang.*;\ni...,19,0,0.553400
1,import java.util.*;\n\npublic class Soltion{\n...,import java.util.*;\n\npublic class mentor1 {\...,20,0,0.634194
2,import java.io.*;\nimport java.util.*;\n\npubl...,import java.io.*;\nimport java.util.*;\n\npubl...,14,1,1.000000
3,\nimport java.util.*;\nimport java.lang.*;\nim...,import java.io.OutputStream;\nimport java.io.I...,6,0,0.886600
4,import java.math.BigDecimal;\nimport java.math...,import java.util.*;\npublic class Sol\n{\n ...,15,0,0.771302
...,...,...,...,...,...
906,import java.io.BufferedReader;\nimport java.io...,\nimport java.io.BufferedReader;\nimport java....,8,1,0.661400
907,import javax.print.attribute.standard.MediaSiz...,\nimport java.util.Scanner;\n\npublic class Me...,4,0,0.789526
908,import java.io.*;\nimport java.util.*;\npublic...,import java.io.BufferedReader;\nimport java.io...,17,0,0.636678
909,import java.util.*;\nimport java.io.*;\n\n\npu...,import java.io.*;\nimport java.util.*;\n\npubl...,15,1,0.761967


In [51]:
data["ResulComp"].min()

0.14040420187824162

In [52]:
def asignar_verdict(level, result):
    if result > level:
        return 1
    else:
        return 0


In [53]:
for i in range(10):
    name = "range>0." + str(i)
    level = i/10
    data[name] = data['ResulComp'].apply(lambda x: asignar_verdict(level,x))
    

In [54]:
total_registros = len(data)
for i in range(10):
    name = "range>0." + str(i)
    coincidencias = (data['verdict'] == data[name]).sum()
    precision = coincidencias / total_registros
    print("Precisión de", name, "es:",precision)


Precisión de range>0.0 es: 0.27552140504939626
Precisión de range>0.1 es: 0.27552140504939626
Precisión de range>0.2 es: 0.27991218441273324
Precisión de range>0.3 es: 0.28869374313940727
Precisión de range>0.4 es: 0.29637760702524696
Precisión de range>0.5 es: 0.3227222832052689
Precisión de range>0.6 es: 0.38419319429198684
Precisión de range>0.7 es: 0.4654226125137212
Precisión de range>0.8 es: 0.6311745334796927
Precisión de range>0.9 es: 0.8079034028540066


In [55]:
datap = data

In [56]:
i = 1
for i in range(100):
    datap = datap[datap["ResulComp"]>(i/100)]
    total_registros = len(datap)
    coincidencias = (datap['verdict'] == datap["range>0.9"]).sum()
    precision = coincidencias / total_registros
    print("coincidencias",coincidencias,"total_registros",total_registros)
    print("Precisión de", name, "es:", precision, "con corte en", (i/100))


coincidencias 736 total_registros 911
Precisión de range>0.9 es: 0.8079034028540066 con corte en 0.0
coincidencias 736 total_registros 911
Precisión de range>0.9 es: 0.8079034028540066 con corte en 0.01
coincidencias 736 total_registros 911
Precisión de range>0.9 es: 0.8079034028540066 con corte en 0.02
coincidencias 736 total_registros 911
Precisión de range>0.9 es: 0.8079034028540066 con corte en 0.03
coincidencias 736 total_registros 911
Precisión de range>0.9 es: 0.8079034028540066 con corte en 0.04
coincidencias 736 total_registros 911
Precisión de range>0.9 es: 0.8079034028540066 con corte en 0.05
coincidencias 736 total_registros 911
Precisión de range>0.9 es: 0.8079034028540066 con corte en 0.06
coincidencias 736 total_registros 911
Precisión de range>0.9 es: 0.8079034028540066 con corte en 0.07
coincidencias 736 total_registros 911
Precisión de range>0.9 es: 0.8079034028540066 con corte en 0.08
coincidencias 736 total_registros 911
Precisión de range>0.9 es: 0.8079034028540066

In [38]:
datap

,sub1,sub2,problem,verdict,ResulComp,range>0.0,range>0.1,range>0.2,range>0.3,range>0.4,range>0.5,range>0.6,range>0.7,range>0.8,range>0.9
2,import java.io.*;\nimport java.util.*;\n\npubl...,import java.io.*;\nimport java.util.*;\n\npubl...,14,1,1.000000,1,1,1,1,1,1,1,1,1,1
9,import java.io.*;\nimport java.util.*;\n\npubl...,import java.io.*;\nimport java.util.*;\n\npubl...,14,1,1.000000,1,1,1,1,1,1,1,1,1,1
12,import java.io.BufferedReader;\nimport java.io...,import java.io.BufferedReader;\nimport java.io...,8,1,0.998433,1,1,1,1,1,1,1,1,1,1
26,import java.io.*;\nimport java.util.*;\n\npubl...,import java.io.*;\nimport java.util.*;\n\npubl...,14,1,1.000000,1,1,1,1,1,1,1,1,1,1
28,import java.util.*;\n\npublic class A {\n\n ...,import java.util.*;\n \npublic class A {\n \n ...,10,1,0.999983,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,//package practice.codeforces.round763;\n\nimp...,import java.util.Scanner;\n \npublic class C {...,16,1,0.999111,1,1,1,1,1,1,1,1,1,1
857,//#Rohitpratap311\n//Keep_Calm_And_Stay_Happy\...,import java.io.*;\nimport java.util.*;\npublic...,17,1,0.999649,1,1,1,1,1,1,1,1,1,1
859,import java.io.DataInputStream;\nimport java.i...,import java.io.DataInputStream;\nimport java.i...,11,1,1.000000,1,1,1,1,1,1,1,1,1,1
871,/*============================================...,/*============================================...,9,1,1.000000,1,1,1,1,1,1,1,1,1,1


In [25]:
coincidencias


91

# Comparación idf

In [10]:
import math



# Función para tokenizar y limpiar el texto


def preprocess(text):
    text = text.lower().split()

    return [word.strip(".?!¿,()--—_") for word in text]



# Calcular la frecuencia de término (TF)


def calculate_tf(tokens):
    tf = {}
    total_words = len(tokens)
    for word in tokens:
        tf[word] = tf.get(word, 0) + 1 / total_words
    return tf

# Calcular el IDF


def calculate_idf(tokens1, tokens2):
    idf = {}
    total_docs = 2
    unique_words = set(tokens1 + tokens2)
    for word in unique_words:
        count = 0
        if word in tokens1:
            count += 1
        if word in tokens2:
            count += 1
        idf[word] = math.log((total_docs / (count+1)))+1
    return idf

# Calcular el vector TF-IDF


def calculate_tfidf(tokens, tf, idf):
    tfidf = {}
    for word in tokens:
        tfidf[word] = tf[word] * idf[word]
    return tfidf


def cosine_similarity(vector_a, vector_b):
    dot_product = np.dot(vector_a, vector_b)
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    cosine_similarity = dot_product / (norm_a * norm_b)
    return cosine_similarity


In [11]:
dataidf = pd.read_csv('labels_clean.csv')


In [12]:
def diferencia_cos_idf(t1,t2):
    texto1 = t1
    texto2 = t2
    tokens1 = preprocess(texto1)
    tokens2 = preprocess(texto2)

    # Calcular el TF-IDF para cada texto
    tf1 = calculate_tf(tokens1)
    tf2 = calculate_tf(tokens2)
    idf = calculate_idf(tokens1, tokens2)
    tfidf1 = calculate_tfidf(tokens1, tf1, idf)
    tfidf2 = calculate_tfidf(tokens2, tf2, idf)

    # Crear la lista de datos para el DataFrame
    data = []
    for word in sorted(set(tokens1 + tokens2), key=lambda x: tokens1.index(x) if x in tokens1 else tokens2.index(x)):
        tfidf1_value = tfidf1.get(word, 0)
        tfidf2_value = tfidf2.get(word, 0)
        data.append([word, tf1.get(word, 0), tf2.get(word, 0),
                    idf.get(word, 0), tfidf1_value, tfidf2_value])

    # Crear el DataFrame
    df = pd.DataFrame(data, columns=["Palabra", "Aparición en texto 1",
                    "Aparición en texto 2", "IDF", "TF(texto 1) * IDF", "TF(texto 2) * IDF"])
    
    # Ejemplo de uso
    vector1 = np.array(df['TF(texto 1) * IDF'])
    vector2 = np.array(df['TF(texto 2) * IDF'])

    distance = cosine_similarity(vector1, vector2)

    return distance
    
    


In [13]:
dataidf['ResulComp'] = dataidf.apply(
    lambda row: diferencia_cos_idf(row['sub1'], row['sub2']), axis=1)


In [14]:
for i in range(10):
    name = "range>0." + str(i)
    level = i/10
    dataidf[name] = dataidf['ResulComp'].apply(
        lambda x: asignar_verdict(level, x))


In [15]:
total_registros = len(dataidf)
for i in range(10):
    name = "range>0." + str(i)
    coincidencias = (dataidf['verdict'] == dataidf[name]).sum()
    precision = coincidencias / total_registros
    print("Precisión de", name, "es:", precision)


Precisión de range>0.0 es: 0.27552140504939626
Precisión de range>0.1 es: 0.27552140504939626
Precisión de range>0.2 es: 0.27552140504939626
Precisión de range>0.3 es: 0.27552140504939626
Precisión de range>0.4 es: 0.27552140504939626
Precisión de range>0.5 es: 0.27552140504939626
Precisión de range>0.6 es: 0.27552140504939626
Precisión de range>0.7 es: 0.27661909989023054
Precisión de range>0.8 es: 0.287596048298573
Precisión de range>0.9 es: 0.43798024149286496
